In [1]:
import os
from pathlib import Path
import sys

# !IMPORTANT!, else Hydra isnt working !
current_path = os.getcwd()
sys.path.insert(0, Path(current_path).absolute().parents[0].as_posix())

import cv2
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
import mediapy
import numpy as np
import torch
from uha import make_pytorch_oxe_iterable_dataset, get_octo_dataset_tensorflow, get_single_dataset_tensorflow
from torchvision.transforms.functional import convert_image_dtype
import omegaconf
from omegaconf import OmegaConf

/home/marcelr/miniforge3/envs/uha_test_policy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-21 14:44:29.324553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 14:44:29.324617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 14:44:29.326659: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 14:44:29.336374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This

## Load Data
Next, we will load a trajectory from the Bridge dataset. We will use the one defined in uha/data/oxe/oxe_dataset_configs.py.

In [2]:
cfg = omegaconf.OmegaConf.load("../uha/data/conf/uha_default_load_config.yaml")
del cfg.interleaved_dataset_cfg.frame_transform_kwargs.image_augment_kwargs # comment out to get augmented data
cfg_transforms = omegaconf.OmegaConf.load("../uha/data/conf/transforms/oxe_no_remapping.yaml")

In [3]:
# dataset = get_octo_dataset_tensorflow(cfg, train=True)
dataset = get_single_dataset_tensorflow(cfg, train=True) # .repeat().unbatch()
is_single_dataset = True
batch_size = 128
# create Pytorch Train Dataset
dataloader = make_pytorch_oxe_iterable_dataset(dataset, train=True, batch_size=batch_size, transform_dict=cfg_transforms, num_workers=0, pin_memory=True, is_single_dataset=is_single_dataset, main_process=False)
# print("len in colab", len(dataloader.dataset))
it = iter(dataloader)
batch = next(it)
batch = next(it)
batch = next(it)
images, image_secondary, image_wrist = [], [], []
for i in range(batch_size):
  images.append(batch["observation"]["image_primary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  image_secondary.append(batch["observation"]["image_secondary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  # image_wrist.append(batch["observation"]["image_wrist"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  print("action", batch["action"][i, :, 0, :])

mediapy.show_video(images, fps=10)
mediapy.show_video(image_secondary, fps=10)
# mediapy.show_video(image_wrist, fps=10)

########################################
constructing single val dataset: bridge_dataset
########################################


2024-07-21 14:44:34.070418: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Cause: Unable to locate the source code of <function _gcd_import at 0x7f3b0704b400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f3b0704b400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f3b0704b400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-07-21 14:44:35.467241: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-07-21 14:44:42.360330: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12987324416 bytes after encountering the first element of size 12682934 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
/home/marcelr/miniforge3/envs/uha_test_policy/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py:222: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_nump

action tensor([[-0.5277,  0.5294,  0.1657, -0.0745, -0.3033,  0.2370,  1.0000]])
action tensor([[-0.7373,  0.7312, -0.1010,  0.2490, -0.0434,  0.2677,  1.0000]])
action tensor([[-0.7767,  0.8667, -0.1608,  1.0000, -0.1807, -0.0010,  1.0000]])
action tensor([[-0.3696,  0.6553, -0.4127,  0.6664, -0.3126, -0.1161,  1.0000]])
action tensor([[-0.2635,  0.3553, -0.6487,  1.0000, -0.1279, -0.2010,  1.0000]])
action tensor([[ 0.0462,  0.3479, -0.7207,  0.4464, -0.0650,  0.0491,  1.0000]])
action tensor([[ 0.0677,  0.1373, -0.7524,  0.5060,  0.4751, -0.0176,  1.0000]])
action tensor([[ 1.5256e-01,  1.0400e-01, -5.1992e-01,  5.0473e-04,  4.4299e-01,
          2.9208e-02,  1.0000e+00]])
action tensor([[ 0.2009,  0.0923, -0.3569,  0.0193,  0.3722, -0.0064,  1.0000]])
action tensor([[ 0.1520,  0.1310, -0.2558, -0.0812,  0.1560,  0.0528,  1.0000]])
action tensor([[ 0.3550,  0.2603, -0.2781,  0.0776,  0.1982,  0.0226,  1.0000]])
action tensor([[ 0.0157,  0.0161, -0.2192,  0.0201,  0.1050,  0.0188,  1

This browser does not support the video tag.


This browser does not support the video tag.
